In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
df = pd.read_pickle('../test-data/numerics_df.pkl')

In [6]:
import matplotlib.pyplot as plt 
plt.ioff()
from typing import Tuple
import seaborn as sns
def plot_single_layer_temporal_heat_map(
        log: pd.DataFrame, 
        layer_name: str, 
        fig_title: str = None, 
        interval: int =  50, 
        figsize: Tuple[int,int] = (20,10)):
    """
        nrows: total_its // interval
        ncols: Num tensor types you're logging (i.e. gradients, activations, weights, optimiser_state)


        TO-DO: 
            Global colour legend & unify scales
    """
    which_tensor = ('metadata', 'grad') # to index for activations vs. gradients
    which_it = ('metadata', 'step')
    which_layer = ('metadata', 'name')

    assert layer_name in log[which_layer].unique(), f'{layer_name} is not in the logs provided'
    assert np.array_equal(log[which_it].unique(), log[log[which_layer] == layer_name][which_it].unique()) == True, f'{layer_name} has not been logged for each training iteration' # perhaps this should be change to ensure all interval are in the steps column

    mod_log = log[log[which_layer] == layer_name] # df for layer in question only

    final_it = mod_log[which_it].max()
    assert final_it % interval == 0, 'Interval must be a multiple of total number of iterations'
    n_rows = final_it // interval + 1 # to ensure final step is displayed.
    # should probably do some soft warning if figsize[1] / n_rows is below some threshold for certain plots
    tensors_logged = mod_log[which_tensor].unique()
    # intialise sub plots
    fig, axs = plt.subplots(ncols=len(tensors_logged), nrows=n_rows, figsize=figsize, sharex='col', sharey='row')

    # loop for generating subplots\
    # iterate over rows
    for it,ax in zip(range(0,final_it+1, interval),axs):

        mod_it_log = mod_log[mod_log[which_it] == it] #filter df for interval it
        # iterate over columns
        for j,col in enumerate(ax):
            # logic for column titles:
            tensor_tbd = tensors_logged[j]
            if it == 0:
                title = 'Activations'
                if tensor_tbd:
                    title = 'Gradients'
                col.set_title(title)

            # draw subplot
            mod_it_tensor_log = mod_it_log[mod_it_log[which_tensor] == tensor_tbd]
            denom = mod_it_tensor_log.exponent_count.sum(axis =1)
            normed_mod_it_tensor_log = mod_it_tensor_log.exponent_count.apply(lambda x: x/denom)

            # temp fix to order exponent column
            col_list = normed_mod_it_tensor_log.columns.to_list()
            col_list = col_list[-2:-1] + col_list[:-2] + col_list[-1:]

            # generate some plot -> 
            sns.heatmap(normed_mod_it_tensor_log[col_list],ax=col)
            col.set_yticks([])
            col.set_ylabel(f'n={it}')
    return fig

# plot_single_layer_temporal_heat_map(df,layer_name='tok_embeddings',interval=100)

In [ ]:
# plots to implement...
    # modules (across layers) with histograms averages

In [26]:
temp =df.query("@df.metadata.name.str.contains('.attention.wk$')").pipe(
    lambda x : x[x.metadata.step == 0]).pipe(lambda x: x[x.metadata.grad == False]).groupby(
        df.metadata.type).exponent_count.sum()

In [30]:
temp.exponent_count.sum(axis=1)

type
torch.nn.modules.linear.Linear    6193152
dtype: int64